In [50]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter 

In [3]:
allmodis = pd.read_csv(r'..\Data\Raw\archive_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight', 'type'], 1)
allviirs = pd.read_csv(r'..\Data\Raw\archive_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'type'], 1)

In [5]:
allmodis.head()

,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp
0,-16.4439,143.5076,319.9,4.5,2.0,2000-11-01,59,300.6,62.8
1,-16.4447,143.5152,321.0,4.5,2.0,2000-11-01,62,300.5,71.6
2,-18.4459,144.8904,324.6,3.2,1.7,2000-11-01,74,303.1,77.4
3,-18.4401,144.8603,317.7,3.3,1.7,2000-11-01,61,303.2,38.5
4,-18.6804,145.5470,317.3,2.9,1.6,2000-11-01,54,299.4,32.8


In [6]:
allmodis.to_csv(r'..\Data\Clean\carchive_M6.csv', index = False)
allviirs.to_csv(r'..\Data\Clean\carchive_V1.csv', index = False)

In [7]:
nrtmodis = pd.read_csv(r'..\Data\Raw\nrt_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight'], 1)
nrtviirs = pd.read_csv(r'..\Data\Raw\nrt_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight'], 1)

In [8]:
nrtmodis.to_csv(r'..\Data\Clean\cnrt_M6.csv', index = False)
nrtviirs.to_csv(r'..\Data\Clean\cnrt_V1.csv', index = False)

In [134]:
### here starts rain.data ###
###
###

In [9]:
rain = pd.read_csv(r'..\Data\Raw\rain.csv')

In [10]:
rain.head()

,Rain ID,Date/Time,Nearest town,State,Latitude,Longitude,Severity confidence description,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Water course,Damage total,Insurance payout,Number of deaths,Number of injuries,Unnamed: 16
0,989,2006-01-06 05:05:00,walcha,NSW,-30.93,151.72,Estimate of rainfall (eg. from filling of rece...,25.0,10.0,35,0,NaN,0.0,0.0,0.0,0.0,NaN
1,990,2006-01-07 01:00:00,rosebank,NSW,-28.64,153.41,Other measurement of rainfall.,0.0,0.0,505,0,NaN,0.0,0.0,0.0,0.0,NaN
2,991,2006-01-12 05:15:00,Walcha,NSW,-30.99,151.59,Other measurement of rainfall.,54.0,35.0,54,0,NaN,0.0,0.0,0.0,0.0,NaN
3,992,2006-01-15 05:20:00,Forbes,NSW,-33.39,148.01,Other measurement of rainfall.,53.0,60.0,53,0,NaN,0.0,0.0,0.0,0.0,NaN
4,993,2006-01-15 21:54:00,Goulburn,NSW,-34.75,149.73,Bureau measurement of rainfall.,85.0,240.0,85,0,NaN,0.0,0.0,0.0,0.0,NaN


In [11]:
rain = rain.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})

In [12]:
rain.nunique()

Rain ID                            3466
Date/Time                          2483
Nearest town                       2149
State                                 7
latitude                           2183
longitude                          2133
Severity confidence description       4
Intense precipitation amount        402
Intense precipitation period         84
Total precipitation                 195
Total precipitation duration         18
Water course                        102
Damage total                          5
Insurance payout                      1
Number of deaths                      4
Number of injuries                    1
Unnamed: 16                           0
dtype: int64

In [13]:
rain['Date/Time'] = pd.to_datetime(rain['Date/Time'])

In [14]:
#spaltet date und time
rain['acq_date'] = [i.date() for i in rain['Date/Time']]
rain['acq_time'] = [i.time() for i in rain['Date/Time']]

In [15]:
rain = rain.drop(['Unnamed: 16', 'Severity confidence description', 'Water course', 'Number of injuries', 'Number of deaths', 'Damage total', 'Damage total', 'Date/Time'], 1)

In [16]:
# adding a new column with the countryname of the coordinate points
rain['region'] = [i['cc'] for i in rg.search(list(zip(*[np.array(rain.latitude), np.array(rain.longitude)])))]

Loading formatted geocoded file...


In [18]:
# looking at the different countrys and their counts
rain.groupby('region').count()

,Rain ID,Nearest town,State,latitude,longitude,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Insurance payout,acq_date,acq_time
region,,,,,,,,,,,,
AU,3416,3415,3416,3416,3416,3416,3416,3416,3416,508,3416,3416
CX,1,1,1,1,1,1,1,1,1,0,1,1
GH,48,47,48,48,48,48,48,48,48,0,48,48
GQ,1,1,1,1,1,1,1,1,1,1,1,1


In [19]:
# selecting only the entries for Australia
rain = rain[rain['region'] == 'AU']

In [20]:
# checking if everything went ok
rain.groupby('region').count()

,Rain ID,Nearest town,State,latitude,longitude,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Insurance payout,acq_date,acq_time
region,,,,,,,,,,,,
AU,3416,3415,3416,3416,3416,3416,3416,3416,3416,508,3416,3416


In [21]:
# overriding the (now useless) region column with the region of Australia
rain['region'] = [i['admin1'] for i in rg.search(list(zip(*[np.array(rain.latitude), np.array(rain.longitude)])))]

In [22]:
rg.search((rain.latitude[0], rain.longitude[0]))

[OrderedDict([('lat', '-30.73333'),
              ('lon', '151.8'),
              ('name', 'Enmore'),
              ('admin1', 'New South Wales'),
              ('admin2', 'Armidale Dumaresq'),
              ('cc', 'AU')])]

In [133]:
### rain map ###
####### ########

In [95]:
import folium
import folium.plugins as plugins
f = folium.Figure(width=750, height=500)
center_lat = -24.003249 
center_long = 133.737310
m = folium.Map(location=[center_lat,center_long], control_scale=True, zoom_start=4,width=750, height=500,zoom_control=True).add_to(f)
for i in range(0,rain.shape[0]):    
    location=[rain.iloc[i]['latitude'], rain.iloc[i]['longitude']]
    folium.CircleMarker(location,radius=1,color='red').add_to(m)
#data_heat = df2[['latitude','longitude','frp']].values.tolist()
#plugins.HeatMap(data_heat).add_to(m)
display(m)

In [80]:
rain.to_csv(r'..\Data\Clean\rain.csv', index = False)

In [ ]:
### here starts wind.data ###
###
###

In [81]:
wind = pd.read_csv(r'..\Data\Raw\wind.csv')

In [82]:
wind.head()

,Wind ID,Date/Time,Duration,Latitude,Longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,...,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale,Damage total,Insurance payout,Number of deaths,Number of injuries,Unnamed: 20
0,1376,2006-01-01 03:32:00,0,-36.88,149.24,Bombala,NSW,AWS,48,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
1,1377,2006-01-01 04:44:00,0,-35.94,148.38,Cabramurra,NSW,AWS,63,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
2,1378,2006-01-01 05:16:00,0,-36.23,149.12,Cooma,NSW,AWS,56,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
3,1379,2006-01-06 04:30:00,0,-30.93,151.72,walcha,NSW,SPOTTER,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
4,1380,2006-01-06 08:45:00,0,-28.85,153.05,Casino,NSW,AWS,54,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN


In [83]:
wind = wind.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})

In [84]:
wind.nunique()

Wind ID                    2833
Date/Time                  2715
Duration                     61
latitude                   1246
longitude                  1258
Nearest town               1146
State                         7
Source                      461
Max Gust speed               76
Max Gust direction          267
Max Mean Wind speed          45
Max Mean Wind direction      46
Path length                   7
Path width                    7
Path direction                8
Fujita scale                  3
Damage total                  3
Insurance payout              1
Number of deaths              2
Number of injuries            2
Unnamed: 20                   0
dtype: int64

In [85]:
wind['Date/Time'] = pd.to_datetime(wind['Date/Time'])

In [86]:
wind['acq_date'] = [i.date() for i in wind['Date/Time']]
wind['acq_time'] = [i.time() for i in wind['Date/Time']]

In [87]:
wind = wind.drop(['Unnamed: 20', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total', 'Date/Time'], 1)

In [88]:
# adding a new column with the countryname of the coordinate points
wind['region'] = [i['cc'] for i in rg.search(list(zip(*[np.array(wind.latitude), np.array(wind.longitude)])))]

In [89]:
# looking at the different countrys and their counts
wind.groupby('region').count()

,Wind ID,Duration,latitude,longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,Max Mean Wind speed,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale,acq_date,acq_time
region,,,,,,,,,,,,,,,,,
AU,2810,2810,2810,2810,2808,2810,1399,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810
GH,21,21,21,21,19,21,14,21,21,21,21,21,21,21,21,21,21
NF,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [90]:
# selecting only the entries for Australia
wind = wind[wind['region'] == 'AU']

In [91]:
# checking if everything went ok
wind.groupby('region').count()

,Wind ID,Duration,latitude,longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,Max Mean Wind speed,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale,acq_date,acq_time
region,,,,,,,,,,,,,,,,,
AU,2810,2810,2810,2810,2808,2810,1399,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810


In [92]:
# overriding the (now useless) region column with the region of Australia
wind['region'] = [i['admin1'] for i in rg.search(list(zip(*[np.array(wind.latitude), np.array(wind.longitude)])))]

In [113]:
import folium
import folium.plugins as plugins
f = folium.Figure(width=750, height=500)
center_lat = -24.003249 
center_long = 133.737310
m = folium.Map(location=[center_lat,center_long], control_scale=True, zoom_start=4,width=750, height=500,zoom_control=True).add_to(f)
# selecting only the wind values from the New South Wales regions
for i in range(0,wind[wind.region == 'New South Wales'].shape[0]):    
    location=[wind[wind.region == 'New South Wales'].iloc[i]['latitude'], wind[wind.region == 'New South Wales'].iloc[i]['longitude']]
    folium.CircleMarker(location,radius=1,color='red').add_to(m)
#data_heat = df2[['latitude','longitude','frp']].values.tolist()
#plugins.HeatMap(data_heat).add_to(m)
display(m)

In [114]:
wind.to_csv(r'..\Data\Clean\wind.csv', index = False)

In [131]:
### here starts lightning.data ###
###
###

In [115]:
lightning = pd.read_csv(r'..\Data\Raw\lightning.csv')

In [36]:
lightning.head()

,Lightning ID,Date/Time,Latitude,Longitude,Nearest town,State,Source,Severity confidence description,Max current,Flash rate,Damage total,Insurance payout,Number of deaths,Number of injuries,Power line deaths
0,242,2006-03-01 00:00:00,-35.0180,117.8840,Albany,WA,Albany Extra,NaN,0,0,0.0,0.0,0.0,0.0,0.0
1,266,2009-10-05 04:00:00,-34.5067,150.7929,DAPTO,NSW,NaN,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
2,267,2009-11-17 07:00:00,-28.6651,153.2845,THE CHANNON,NSW,Spotter,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
3,269,2011-03-11 03:40:00,-31.1095,150.9147,SOUTH TAMWORTH,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN
4,270,2011-02-28 08:15:00,-31.9652,141.4513,BROKEN HILL,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN


In [120]:
lightning = lightning.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})

In [24]:
lightning.nunique()

Lightning ID                       20
Date/Time                          19
Latitude                           18
Longitude                          18
Nearest town                       18
State                               5
Source                              7
Severity confidence description     3
Max current                         1
Flash rate                          3
Damage total                        1
Insurance payout                    1
Number of deaths                    1
Number of injuries                  1
Power line deaths                   1
Unnamed: 15                         0
dtype: int64

In [116]:
lightning['Date/Time'] = pd.to_datetime(lightning['Date/Time'])

In [117]:
lightning['acq_date'] = [i.date() for i in lightning['Date/Time']]
lightning['acq_time'] = [i.time() for i in lightning['Date/Time']]

In [118]:
lightning = lightning.drop(['Unnamed: 15', 'Power line deaths', 'Date/Time', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total', 'Flash rate', 'Max current', 'Severity confidence description'], 1)

In [122]:
# adding a new column with the countryname of the coordinate points
lightning['region'] = [i['cc'] for i in rg.search(list(zip(*[np.array(lightning.latitude), np.array(lightning.longitude)])))]

In [123]:
# selecting only the entries for Australia
lightning = lightning[lightning['region'] == 'AU']

In [124]:
# overriding the (now useless) region column with the region of Australia
lightning['region'] = [i['admin1'] for i in rg.search(list(zip(*[np.array(lightning.latitude), np.array(lightning.longitude)])))]

In [125]:
import folium
import folium.plugins as plugins
f = folium.Figure(width=750, height=500)
center_lat = -24.003249 
center_long = 133.737310
m = folium.Map(location=[center_lat,center_long], control_scale=True, zoom_start=4,width=750, height=500,zoom_control=True).add_to(f)
for i in range(0,lightning.shape[0]):    
    location=[lightning.iloc[i]['latitude'], lightning.iloc[i]['longitude']]
    folium.CircleMarker(location,radius=1,color='red').add_to(m)
#data_heat = df2[['latitude','longitude','frp']].values.tolist()
#plugins.HeatMap(data_heat).add_to(m)
display(m)

In [126]:
lightning.to_csv(r'..\Data\Clean\lightning.csv', index = False)

In [6]:
nrtmodis['acq_date'] = pd.to_datetime(nrtmodis['acq_date'])
df = nrtmodis.groupby([pd.Grouper(key='acq_date', freq='M')])['brightness'].mean().reset_index().sort_values('acq_date')

In [11]:
df = nrtmodis.groupby([pd.Grouper(key='acq_date', freq='M')])['brightness'].mean().reset_index().sort_values('acq_date')
#bla = nrtmodis.groupby('acq_date',pd.Grouper(freq = 'M')).mean()

In [12]:
df

,acq_date,brightness
0,2020-01-31,341.666352
1,2020-02-29,339.083109
2,2020-03-31,326.820262
3,2020-04-30,325.316150
4,2020-05-31,323.824857


In [39]:
list(zip(*[np.array(rain.Latitude), np.array(rain.Longitude)]))

[(-30.93, 151.72),
 (-28.64, 153.41),
 (-30.99, 151.59),
 (-33.39, 148.01),
 (-34.75, 149.73),
 (-35.28, 149.12),
 (-33.6968, 150.5685),
 (-28.5207, 153.5401),
 (-28.52, 153.54),
 (-30.32, 153.12),
 (-30.32, 153.12),
 (-29.49, 153.24),
 (-35.23, 149.01),
 (-33.42, 149.59),
 (-29.5, 151.3),
 (-33.75, 150.68),
 (-33.84, 150.85),
 (-33.64, 151.33),
 (-33.69, 151.1096),
 (-33.74, 151.13),
 (-33.74, 151.13),
 (-33.28, 151.57),
 (-33.43, 151.34),
 (-28.65, 153.62),
 (-32.782, 152.085),
 (-33.729, 150.954),
 (-35.32, 149.12),
 (-32.77, 151.08),
 (-33.6, 149.4),
 (-30.22, 151.67),
 (-30.215, 151.668),
 (-32.265, 150.885),
 (-28.39, 153.19),
 (-33.89, 150.91),
 (-33.865, 150.951),
 (-30.441, 151.353),
 (-29.932, 152.718),
 (-35.36, 149.23),
 (-34.74, 148.89),
 (-33.697, 150.568),
 (-31.45, 142.13),
 (-31.43, 147.27),
 (-33.56, 150.218),
 (-31.45, 142.13),
 (-31.15, 150.985),
 (-32.97, 151.7),
 (-33.42, 149.58),
 (-32.099, 152.388),
 (-31.432, 152.485),
 (-31.18, 151.43),
 (-33.42, 149.58),
 (-3

In [129]:
### here starts weather.data ###
# Heading 1
# Heading 2
## Heading 2.1
## Heading 2.2

In [23]:
weather = pd.read_csv(r'..\Data\Raw\weatherAUS.csv')

In [24]:
weather.keys()

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [26]:
weather['Location']

0         Albury
1         Albury
2         Albury
3         Albury
4         Albury
           ...  
142188     Uluru
142189     Uluru
142190     Uluru
142191     Uluru
142192     Uluru
Name: Location, Length: 142193, dtype: object

In [47]:
weatherloc = weather.Location.unique()
weatherloc = pd.DataFrame(weatherloc, columns=['Location']) 

In [104]:
from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="fintu-blog-geocoding-python")
bla1 , bla2 = gc.geocode("Pears, Australia")[1]

In [102]:
print(bla1, bla2)

-27.4752488 152.9920742


In [110]:
#applying geocoordinates to locations
from geopy.extra.rate_limiter import RateLimiter

def rate_limited_geocode(query):
    geocode = RateLimiter(gc.geocode, min_delay_seconds=1)
    return geocode(query)
#geocode der 2ten funktion ist eine andere variable
def geocodeb(row):
    lookup_query = row["Location"] + ", " + "Australia"
    lookup_result = rate_limited_geocode(lookup_query)
    return lookup_result
weatherloc["loctemp"] = weatherloc.apply(geocodeb, axis=1)

In [66]:
latitude = weatherloc['loctemp'][0]

(-36.0804766, 146.9162795)

In [112]:
weatherloc[['a','b','latitude','longitude','bla']] = pd.DataFrame(weatherloc.loctemp.tolist(), columns= ['a','b','latitude','longitude','bla'])

ValueError: 5 columns passed, passed data had 2 columns

In [114]:
tempbla = [i [1] for i in weatherloc.loctemp]

In [113]:
weatherloc.loctemp.tolist()

[Location(Albury, Albury City Council, 2640, Australia, (-36.0804766, 146.9162795, 0.0)),
 Location(Badgerys Creek, Eurobodalla Shire Council, Australia, (-35.8348792, 149.9957662, 0.0)),
 Location(Cobar, Cobar Shire Council, 2835, Australia, (-31.4983333, 145.8344444, 0.0)),
 Location(Coffs Harbour, Coffs Harbour City Council, Australia, (-30.2962407, 153.1135293, 0.0)),
 Location(Moree, Moree Plains Shire Council, 2400, Australia, (-29.4617202, 149.8407153, 0.0)),
 Location(Newcastle, Newcastle-Maitland, Newcastle City Council, 2300, Australia, (-32.9272881, 151.7812534, 0.0)),
 Location(Norah Head, Central Coast Council, New South Wales, 2263, Australia, (-33.2816667, 151.5677778, 0.0)),
 Location(Norfolk Island, Australia, (-29.0328038, 167.9483137, 0.0)),
 Location(Penrith, Sydney, Penrith City Council, New South Wales, 2750, Australia, (-33.7510792, 150.6941684, 0.0)),
 Location(Richmond District, Station Street, Fairfield, City of Darebin, Victoria, 3078, Australia, (-37.80745, 

In [119]:
weatherloc['latitude'] , weatherloc['longitude'] = np.array(tempbla).T

In [127]:
weatherloc = weatherloc.drop(['loctemp'],1)

In [128]:
weatherloc

,Location,latitude,longitude
0,Albury,-36.080477,146.916280
1,Badgerys Creek,-35.834879,149.995766
2,Cobar,-31.498333,145.834444
3,Coffs Harbour,-30.296241,153.113529
4,Moree,-29.461720,149.840715
5,Newcastle,-32.927288,151.781253
6,Norah Head,-33.281667,151.567778
7,Norfolk Island,-29.032804,167.948314
8,Penrith,-33.751079,150.694168
9,Richmond,-37.807450,144.990718


In [81]:
import re
alb = []
for i in weatherloc.Location: 
    alb.append(re.findall('[A-Z][^A-Z]*', i ))

In [91]:
alb

[['Albury'],
 ['Badgerys', 'Creek'],
 ['Cobar'],
 ['Coffs', 'Harbour'],
 ['Moree'],
 ['Newcastle'],
 ['Norah', 'Head'],
 ['Norfolk', 'Island'],
 ['Penrith'],
 ['Richmond'],
 ['Sydney'],
 ['Sydney', 'Airport'],
 ['Wagga', 'Wagga'],
 ['Williamtown'],
 ['Wollongong'],
 ['Canberra'],
 ['Tuggeranong'],
 ['Mount', 'Ginini'],
 ['Ballarat'],
 ['Bendigo'],
 ['Sale'],
 ['Melbourne', 'Airport'],
 ['Melbourne'],
 ['Mildura'],
 ['Nhil'],
 ['Portland'],
 ['Watsonia'],
 ['Dartmoor'],
 ['Brisbane'],
 ['Cairns'],
 ['Gold', 'Coast'],
 ['Townsville'],
 ['Adelaide'],
 ['Mount', 'Gambier'],
 ['Nuriootpa'],
 ['Woomera'],
 ['Albany'],
 ['Witchcliffe'],
 ['Pearce', 'R', 'A', 'A', 'F'],
 ['Perth', 'Airport'],
 ['Perth'],
 ['Salmon', 'Gums'],
 ['Walpole'],
 ['Hobart'],
 ['Launceston'],
 ['Alice', 'Springs'],
 ['Darwin'],
 ['Katherine'],
 ['Uluru']]

In [100]:
ergebnis = []
for i in alb:
    trennzeichen = ' '
    ergebnis.append(trennzeichen.join(i))
print(ergebnis)


['Albury', 'Badgerys Creek', 'Cobar', 'Coffs Harbour', 'Moree', 'Newcastle', 'Norah Head', 'Norfolk Island', 'Penrith', 'Richmond', 'Sydney', 'Sydney Airport', 'Wagga Wagga', 'Williamtown', 'Wollongong', 'Canberra', 'Tuggeranong', 'Mount Ginini', 'Ballarat', 'Bendigo', 'Sale', 'Melbourne Airport', 'Melbourne', 'Mildura', 'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns', 'Gold Coast', 'Townsville', 'Adelaide', 'Mount Gambier', 'Nuriootpa', 'Woomera', 'Albany', 'Witchcliffe', 'Pearce R A A F', 'Perth Airport', 'Perth', 'Salmon Gums', 'Walpole', 'Hobart', 'Launceston', 'Alice Springs', 'Darwin', 'Katherine', 'Uluru']


In [98]:
trennzeichen = ' '
ergebnis = trennzeichen.join(alb[1])
print(ergebnis)

Badgerys Creek


In [105]:
weatherloc['Location'] = ergebnis

In [109]:
weatherloc.Location[38] = 'Pearce'
weatherloc.Location[38]

'Pearce'

In [135]:
## maske ##
mask = weather.Location == 'Albury'

In [136]:
mask

0          True
1          True
2          True
3          True
4          True
          ...  
142188    False
142189    False
142190    False
142191    False
142192    False
Name: Location, Length: 142193, dtype: bool

In [151]:
weather[mask][['latitude','longitude']] = weatherloc[weatherloc.Location == 'Albury'][['latitude','longitude']]

D:\PythonData\Anacondas\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [154]:
weather

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,latitude,longitude
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.1,8.0,NaN,16.9,21.8,No,0.0,No,0,0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1007.8,NaN,NaN,17.2,24.3,No,0.0,No,0,0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1008.7,NaN,2.0,21.0,23.2,No,0.0,No,0,0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1012.8,NaN,NaN,18.1,26.5,No,1.0,No,0,0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,No,0.2,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,ESE,...,1021.2,NaN,NaN,9.4,20.9,No,0.0,No,0,0
142189,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,1020.3,NaN,NaN,10.1,22.4,No,0.0,No,0,0
142190,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,1019.1,NaN,NaN,10.9,24.5,No,0.0,No,0,0
142191,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,1016.8,NaN,NaN,12.5,26.1,No,0.0,No,0,0


In [157]:
weather.loc[mask,'latitude'] = weatherloc[weatherloc.Location == 'Albury']['latitude']
weather.loc[mask,'longitude'] = weatherloc[weatherloc.Location == 'Albury']['longitude']

In [153]:
weatherloc[weatherloc.Location == 'Albury'][['latitude','longitude']]

,latitude,longitude
0,-36.080477,146.91628


In [158]:
weather

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,latitude,longitude
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.1,8.0,NaN,16.9,21.8,No,0.0,No,-36.080477,146.91628
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1007.8,NaN,NaN,17.2,24.3,No,0.0,No,NaN,NaN
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1008.7,NaN,2.0,21.0,23.2,No,0.0,No,NaN,NaN
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1012.8,NaN,NaN,18.1,26.5,No,1.0,No,NaN,NaN
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,No,0.2,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,ESE,...,1021.2,NaN,NaN,9.4,20.9,No,0.0,No,0.000000,0.00000
142189,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,1020.3,NaN,NaN,10.1,22.4,No,0.0,No,0.000000,0.00000
142190,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,1019.1,NaN,NaN,10.9,24.5,No,0.0,No,0.000000,0.00000
142191,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,1016.8,NaN,NaN,12.5,26.1,No,0.0,No,0.000000,0.00000


In [159]:
weather.loc[mask,'latitude']

0      -36.080477
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
3006          NaN
3007          NaN
3008          NaN
3009          NaN
3010          NaN
Name: latitude, Length: 3011, dtype: float64

In [162]:
weatherloc[weatherloc.Location == 'Albury']['latitude']

0   -36.080477
Name: latitude, dtype: float64

In [187]:
weather['latitude'] = weather.apply(lambda row: weatherloc[weatherloc.Location == 'Albury']['latitude'] if row ['Location'] == 'Albury')

SyntaxError: invalid syntax (<ipython-input-187-b0a06287b19c>, line 1)

In [180]:
weather['latitude'] = weather.apply(lambda row: 1)